In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import requests
from requests import get
from time import sleep
import csv
from random import randint

### For single page

In [2]:
url = "https://www.medicalmarijuana.com/directory/categories/marijuana-dispensaries?p=106&category=5&zoom=15&is_mile=0&directory_radius=0&view=list&sort=newest"

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}

results = requests.get(url, headers=headers)

soup = BeautifulSoup(results.text, "html.parser")

#print(soup.prettify())

### For multiple pages

In [16]:
pages = np.arange(1, 109)

titles = []
category = []
emails = []
numb = []
websites = []
location = []
description = []

for page in pages: 
    page = requests.get("https://www.medicalmarijuana.com/directory/categories/marijuana-dispensaries?p="+ str(page) + "&category=5&zoom=15&is_mile=0&directory_radius=0&view=list&sort=newest", headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    #for element in soup.findAll(attrs={'class': 'sabai-directory-contact-email'}):
        #email = element.find('a')
        #emails.append(email.text)
    
    item_list = soup.findAll('div', attrs={'class': 'sabai-col-xs-9 sabai-directory-main'})
    for container in item_list:
        #company
        name = container.a.text
        if name is None:
            titles.append('None')
        else:
            titles.append(name)

        #cateogories
        cat = container.find('div', class_='sabai-directory-category').text
        if cat is None:
            category.append('None')
        else:
            category.append(cat)
    
        #address
        add = container.find('div', class_='sabai-directory-location')
        address = add.find('span').text
        if add is None:
            location.append('None')
        else:
            location.append(address)
    
        #description
        des = container.find('div', class_='sabai-directory-body')
        if des is None:
            description.append('None')
        else:
            description.append(des.text.strip())
            
        num = container.find('span', class_='sabai-visible-xs-inline')
        if num is None:
            numb.append('None')
        else:
            numb.append(num.text.strip())
            
        email = container.find('div', class_='sabai-directory-contact-email')
        if email is None:
            emails.append('None')
        else:
            emails.append(email.text.strip())
            
        website = container.find('div', class_='sabai-directory-contact-website')
        if website is None:
            websites.append('None')
        else:
            websites.append(website.text.strip())

In [24]:
tab=pd.DataFrame()
tab['Company']=titles
tab['Website']=websites
tab['Description']=description
tab['Address']=location
tab['Company Phone']=numb
tab['Email']=emails
tab['Industry Category']=category
tab.head()

,Company,Website,Description,Address,Company Phone,Email,Industry Category
0,Serra Dispensary Belmont,https://shopserra.com/location/southeast-portl...,None,"2519 SE Belmont St, Portland, OR 97214, Unite...",971-544-7055,info@shopserra.com,\n Marijuana Dispensaries
1,Medical Cannabis for patients,http://www.420medications.net,None,"Sacramento, CA, United States",949-334-7402,info@420medications.net,\n Marijuana Dispensaries
2,Ohana Farms,http://NULL,* Safe and discreet medicine delivery service ...,301 1st Bainbridge Island Washington 98110 Ki...,206-902-6326,ohanainstitute@gmail.com,\n Marijuana Dispensaries
3,The Speezy,http://NULL,All American Healing Group is a medical cannab...,6405 s Vermont Los Angeles California 90044 L...,2139732420,thaspeezy@gmail.com,\n Marijuana Dispensaries
4,Seattle Green Care Delivery,http://www.SeattleGreenCare.com,Only the finest quality MMJ – grown indoors by...,Haller Lake Seattle Washington 98125 King,206-501-4063,Info@SeattleGreenCare.com,\n Marijuana Dispensaries
